In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Define the dataset name and version
dataset_name = "squad"
# dataset_version = "wikitext-103-raw-v1"

# Load the dataset from Hugging Face
dataset = load_dataset(dataset_name) #, dataset_version)
train_dataset = dataset['train']
valid_dataset = dataset['validation']

# load train and validation split of squad
# train_dataset  = nlp.load_dataset('squad', split=nlp.Split.TRAIN)
# valid_dataset = nlp.load_dataset('squad', split=nlp.Split.VALIDATION)


# Define the device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the T5 model and tokenizer
# model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
from transformers import  AutoTokenizer, AutoModelWithLMHead, pipeline
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

In [ ]:
activation = {}
def getActivation(name):
  # the hook signature
    def hook(model, input, output):
        if name not in activation.keys():
            activation[name] = [output.detach()]
        else:
            activation[name].append(output.detach())
    return hook

hooks = []
for name, module in model.named_modules():
    if "act" in name:
        hooks.append(module.register_forward_hook(getActivation(name)))

for hook in hooks:
    hook.remove()

In [ ]:
question = "What my mother's name?"
context = "My mother's name is Ann."
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
print(output.size())
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)